In [0]:
# import SparkSession from pyspark.sql and create a SparkSession, which is the entry point to Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('Colab').config('spark.ui.port', '4050').getOrCreate()
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

### 1.What are the employee name and the supervisor's name of each employee?

In [0]:
%sql
SELECT a.employeeName AS Employee_Name,
b.employeeName AS Supervisor_Name
FROM hw5_employee_json a, hw5_employee_json b
WHERE a.employeeSupervisor = b.employeeID 

Employee_Name,Supervisor_Name
Laura Ellenburg,Robert Lewis
Phil Morris,Robert Lewis
Lawrence Haley,Robert Lewis
Robert Lewis,Phil Morris
Mary Smith,Lawrence Haley


### 2.What are the employee name and age when each employee was hired?

In [0]:
%sql
SELECT employeeName,
  round(DATEDIFF( employeeDateHired, employeeBirthDate )/365) AS hired_age   
FROM hw5_employee_json

employeeName,hired_age
Laura Ellenburg,null
Phil Morris,42.0
Lawrence Haley,35.0
Robert Lewis,33.0
Mary Smith,31.0


### 3.How many employees in each city?

In [0]:
%sql
select employeeCity, count(employeeID) AS number_of_employee
from hw5_employee_json
group by employeeCity

employeeCity,number_of_employee
Nashville,2
Knoxville,2
Clearwater,1


### 4.How many skills applied by each employee?

In [0]:
%sql
select e.employeeID, count(s.skillID) as skill_count
from hw5_employee_json e, hw5_employeeskill_json s
where e.employeeID = s.employeeID
group by e.employeeID

employeeID,skill_count
123-44-345,4
334-45-667,2
454-56-768,3
000-00-000,3


### 5.How many employees apply each skill at each work center?

In [0]:
%sql
select s.skillID, w.workCenterID, count(s.employeeID) as emp_count
from hw5_employeeskill_json s, hw5_employeeworkcenter_json w
where s.employeeID = w.employeeID
group by s.skillID, w.workCenterID

skillID,workCenterID,emp_count
BS12,TW1,1
TS10,TW1,1
TS10,SM1,1
BS12,SM1,1
RT1,SM1,1
QC1,SM1,1
RT1,TW1,1


### %Python

In [0]:
%python
df_e = spark.read.table('hw5_employee_json')
df_s = spark.read.table('hw5_employeeskill_json')
df_w = spark.read.table('hw5_employeeworkcenter_json')
df_l = spark.read.table('hw5_skill_json')
df_c = spark.read.table('hw5_workcenter_json')

In [0]:
df_e.printSchema()
df_s.printSchema()
df_w.printSchema()
df_l.printSchema()
df_c.printSchema()

root
 |-- employeeAddress: string (nullable = true)
 |-- employeeBirthDate: string (nullable = true)
 |-- employeeCity: string (nullable = true)
 |-- employeeDateHired: string (nullable = true)
 |-- employeeID: string (nullable = true)
 |-- employeeName: string (nullable = true)
 |-- employeeState: string (nullable = true)
 |-- employeeSupervisor: string (nullable = true)
 |-- employeeZipCode: string (nullable = true)

root
 |-- employeeID: string (nullable = true)
 |-- skillID: string (nullable = true)

root
 |-- employeeID: string (nullable = true)
 |-- workCenterID: string (nullable = true)

root
 |-- skillDescription: string (nullable = true)
 |-- skillID: string (nullable = true)

root
 |-- workCenterID: string (nullable = true)
 |-- workCenterLocation: string (nullable = true)



In [0]:
df_e.show(7)

+--------------------+-----------------+------------+-----------------+----------+---------------+-------------+------------------+---------------+
|     employeeAddress|employeeBirthDate|employeeCity|employeeDateHired|employeeID|   employeeName|employeeState|employeeSupervisor|employeeZipCode|
+--------------------+-----------------+------------+-----------------+----------+---------------+-------------+------------------+---------------+
|5342 Picklied Tro...|             null|   Nashville|       2000-02-22|000-00-000|Laura Ellenburg|           TN|        454-56-768|          38010|
|     2134 Hilltop Rd|       1957-01-05|   Knoxville|       1999-06-12|123-44-345|    Phil Morris|           TN|        454-56-768|          37920|
|5970 Spring Crest Rd|       1963-08-15|   Nashville|       1999-01-05|334-45-667| Lawrence Haley|           TN|        454-56-768|          54545|
|  17834 Deerfield Ln|       1964-08-25|   Knoxville|       1998-01-01|454-56-768|   Robert Lewis|           TN|

In [0]:
df_s.show(7)

+----------+-------+
|employeeID|skillID|
+----------+-------+
|000-00-000|    UC1|
|000-00-000|    US1|
|000-00-000|    UT1|
|123-44-345|   BS12|
|123-44-345|    QC1|
|123-44-345|    RT1|
|123-44-345|   TS10|
+----------+-------+
only showing top 7 rows



In [0]:
df_w.show(7)

+----------+------------+
|employeeID|workCenterID|
+----------+------------+
|123-44-345|         SM1|
|454-56-768|         TW1|
+----------+------------+



In [0]:
df_l.show(7)

+----------------+-------+
|skillDescription|skillID|
+----------------+-------+
|   12in Band Saw|   BS12|
| Quality Control|    QC1|
|          Router|    RT1|
|     Sander-Belt|    SB1|
|  Sander-Orbital|    SO1|
|  10in Table Saw|   TS10|
|  12in Table Saw|   TS12|
+----------------+-------+
only showing top 7 rows



In [0]:
df_c.show(7)

+------------+--------------------+
|workCenterID|  workCenterLocation|
+------------+--------------------+
|         SM1|       Main Saw Mill|
|         TW1|     Tampa Warehouse|
|         WR1|Warehouse and Rec...|
+------------+--------------------+



### 1.What are the employee name and the supervisor's name of each employee

In [0]:
from pyspark.sql.functions import concat, lit, col
df_e.alias("emp1").join(df_e.alias("emp2"), \
    col("emp1.employeeSupervisor") == col("emp2.employeeID"),"inner") \
    .select(col("emp1.employeeName").alias("Employee_Name"), \
      col("emp2.employeeName").alias("Superior_Name")) \
   .show(truncate=False)

+---------------+--------------+
|Employee_Name  |Superior_Name |
+---------------+--------------+
|Laura Ellenburg|Robert Lewis  |
|Phil Morris    |Robert Lewis  |
|Lawrence Haley |Robert Lewis  |
|Robert Lewis   |Phil Morris   |
|Mary Smith     |Lawrence Haley|
+---------------+--------------+



### 2.What are the employee name and age when each employee was hired?

In [0]:
from pyspark.sql.functions import *
df_e.select(
      col("employeeName"),
      round(datediff(col("employeeDateHired"),col("employeeBirthDate"))/365).alias("hired_age")
    ).show()

+---------------+---------+
|   employeeName|hired_age|
+---------------+---------+
|Laura Ellenburg|     null|
|    Phil Morris|     42.0|
| Lawrence Haley|     35.0|
|   Robert Lewis|     33.0|
|     Mary Smith|     31.0|
+---------------+---------+



### 3.How many employees in each city?

In [0]:
df_e.select("employeeCity", "employeeID").groupby("employeeCity").count().withColumnRenamed('count','Number of employees').show()

+------------+-------------------+
|employeeCity|Number of employees|
+------------+-------------------+
|   Nashville|                  2|
|   Knoxville|                  2|
|  Clearwater|                  1|
+------------+-------------------+



### 4.How many skills applied by each employee?

In [0]:
df_e.join(df_s, "employeeID").groupby("employeeID").count().withColumnRenamed('count','skill count').show()

+----------+-----------+
|employeeID|skill count|
+----------+-----------+
|123-44-345|          4|
|334-45-667|          2|
|454-56-768|          3|
|000-00-000|          3|
+----------+-----------+



### 5.How many employees apply each skill at each work center?

In [0]:
df_s.join(df_w, "employeeID").groupby("skillID","workCenterID").count().withColumnRenamed('count','employee count').show()

+-------+------------+--------------+
|skillID|workCenterID|employee count|
+-------+------------+--------------+
|   BS12|         TW1|             1|
|   TS10|         TW1|             1|
|   TS10|         SM1|             1|
|   BS12|         SM1|             1|
|    RT1|         SM1|             1|
|    QC1|         SM1|             1|
|    RT1|         TW1|             1|
+-------+------------+--------------+

